In [21]:
import pandas as pd
PROJECT_NAME = "Sep_21"
DATA_PATH = f"../log/{PROJECT_NAME}/koopman_correlation_results_log.csv"
data = pd.read_csv(DATA_PATH)
data

,env_name,state_dim,u_dim,max_train_samples,train_samples,val_samples,test_samples,Ksteps,train_steps,batch_size,...,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params,m,coeff
0,G1,53,23.0,140000,210,20000,20000,15,20000,64,...,0.292336,0.712445,0.288788,2.830161,0.157042,../log/Sep_21/best_models/20260120_221225_mode...,200148,186474,0,1
1,G1,53,23.0,140000,2104,20000,20000,15,20000,64,...,0.196315,0.240509,0.194622,1.416907,0.069007,../log/Sep_21/best_models/20260120_221407_mode...,200148,186474,0,10
2,G1,53,23.0,140000,21043,20000,20000,15,20000,64,...,0.158649,0.159957,0.157595,0.943719,0.039221,../log/Sep_21/best_models/20260120_221545_mode...,200148,186474,0,100
3,G1,53,23.0,140000,494,20000,20000,15,20000,64,...,0.247192,0.538686,0.244506,11.438308,0.112448,../log/Sep_21/best_models/20260120_221725_mode...,242654,213716,0,1
4,G1,53,23.0,140000,4943,20000,20000,15,20000,64,...,0.175517,0.193621,0.173840,5.295204,0.053817,../log/Sep_21/best_models/20260120_221902_mode...,242654,213716,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Go2,35,12.0,140000,31555,20000,20000,15,20000,64,...,0.214854,0.216588,0.213585,28.762976,0.064012,../log/Sep_21/best_models/20260121_182657_mode...,396941,293936,0,20
296,Go2,35,12.0,140000,63110,20000,20000,15,20000,64,...,0.212093,0.214555,0.210709,31.026777,0.060932,../log/Sep_21/best_models/20260121_182826_mode...,396941,293936,0,40
297,Go2,35,12.0,140000,17718,20000,20000,15,20000,64,...,0.222654,0.223550,0.220786,76.655067,0.068368,../log/Sep_21/best_models/20260121_182959_mode...,799021,437856,0,5
298,Go2,35,12.0,140000,70873,20000,20000,15,20000,64,...,0.210267,0.213103,0.209189,85.507378,0.057650,../log/Sep_21/best_models/20260121_183136_mode...,799021,437856,0,20


In [22]:
data.columns

Index(['env_name', 'state_dim', 'u_dim', 'max_train_samples', 'train_samples',
       'val_samples', 'test_samples', 'Ksteps', 'train_steps', 'batch_size',
       'initial_lr', 'lr_step', 'lr_gamma', 'max_norm', 'val_step', 'gamma',
       'use_residual', 'use_control_loss', 'use_covariance_loss',
       'cov_loss_weight', 'all_loss', 'encode_dim_param', 'encode_dim',
       'encode_dim_mode', 'hidden_layers', 'hidden_dim', 'seed', 'normalize',
       'best_val_Kloss', 'convergence_val_Kloss', 'test_Kloss', 'test_CovLoss',
       'test_ControlLoss', 'model_path', 'num_params', 'encoder_num_params',
       'm', 'coeff'],
      dtype='object')

In [23]:
data['coeff'].unique()

array([  1,  10, 100,   5,  20,  40])

In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# ============================
# Global Settings (match your style)
# ============================
mpl.rcParams.update({
    "figure.dpi": 150,
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "axes.titlesize": 9,
    "axes.labelsize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
    "lines.linewidth": 1.4,
    "lines.markersize": 5.5,
    "axes.formatter.use_mathtext": True,
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
    "axes.prop_cycle": mpl.cycler(color=plt.cm.tab10.colors)
})

# ============================
# Configuration
# ============================
PROJECT_NAME = "Sep_21"
OUT_DIR = PROJECT_NAME
os.makedirs(OUT_DIR, exist_ok=True)

DATA_PATH = f"../log/{PROJECT_NAME}/koopman_correlation_results_log.csv"
assert os.path.exists(DATA_PATH), f"Log CSV not found: {DATA_PATH}"

ENVS  = ["G1", "Go2"]
SEEDS = [17382, 76849, 20965, 84902, 51194]
ENCODE_MULTS = [1, 2, 4, 8, 16]
COEFFS = [1, 5, 10, 20, 40, 100]

# fixed config
LAYER_DEPTHS = [3]
HIDDEN_DIMS  = [256]
USE_RESIDUAL = True
USE_CONTROL_LOSS = True
USE_COV_LOSS = True

# ============================
# Fit helpers
# ============================
def scaling_model(D, A, alpha, C):
    return A * np.power(D, -alpha) + C

def scaling_model_log_fit(D, A, alpha, C):
    pred = scaling_model(D, A, alpha, C)
    pred = np.maximum(pred, 1e-30)  # safety
    return np.log(pred)

def fit_scaling_law_log(D, E):
    """Fit in log-space: minimize || log(E) - log(A D^-alpha + C) ||"""
    D = np.asarray(D, dtype=float)
    E = np.asarray(E, dtype=float)

    mask = (D > 0) & (E > 0)
    D, E = D[mask], E[mask]
    if len(D) < 3:
        return None

    log_D = np.log(D)
    log_E = np.log(E)

    # init: rough slope fit ignoring C
    try:
        slope, intercept = np.polyfit(log_D, log_E, 1)
        alpha_init = -slope
        A_init = np.exp(intercept)
    except:
        alpha_init = 1.0
        A_init = float(np.max(E))

    try:
        popt, _ = curve_fit(
            scaling_model_log_fit,
            D,
            log_E,
            p0=[A_init, alpha_init, 0.0],
            bounds=([0.0, -np.inf, 0.0], [np.inf, np.inf, np.inf]),
            maxfev=20000
        )
        return popt
    except Exception as e:
        print(f"    ⚠️ fit failed: {e}")
        return None

def coeff_display_name(coeff: int) -> str:
    if int(coeff) == 100:
        return "full train sample"
    return f"coeff={int(coeff)}"

# ============================
# Load & Filter
# ============================
log = pd.read_csv(DATA_PATH)

def as_bool(x):
    if isinstance(x, bool):
        return x
    try:
        return bool(int(x))
    except:
        return bool(x)

for c in ["use_residual", "use_control_loss", "use_covariance_loss"]:
    if c in log.columns:
        log[c] = log[c].apply(as_bool)

mask = (
    log["env_name"].isin(ENVS) &
    log["seed"].isin(SEEDS) &
    log["encode_dim_param"].isin(ENCODE_MULTS) &
    log["hidden_layers"].isin(LAYER_DEPTHS) &
    log["hidden_dim"].isin(HIDDEN_DIMS) &
    (log["use_residual"] == USE_RESIDUAL) &
    (log["use_control_loss"] == USE_CONTROL_LOSS) &
    (log["use_covariance_loss"] == USE_COV_LOSS)
)

if "coeff" in log.columns:
    mask &= log["coeff"].isin(COEFFS)

filtered = log[mask].copy()
assert not filtered.empty, "Filtered dataframe is empty — check your mask / columns."

filtered["coeff"] = filtered["coeff"].astype(int)

# We still fit using z_dim, but label x-axis as "Encode Dimension"
filtered["z_dim"] = filtered["state_dim"].astype(float) + filtered["encode_dim"].astype(float)

# ============================
# Plot: Error vs Encode Dimension (internally z_dim)
# ============================
print("\n--- Plot: Error vs Encode Dimension (seed dots + fit by coeff) ---")
fig, axes = plt.subplots(1, 2, figsize=(12, 3.8), sharey=False)

for ax, env in zip(axes, ENVS):
    ax.set_title(env)
    sub_env = filtered[filtered["env_name"] == env].copy()

    for coeff in sorted(sub_env["coeff"].unique()):
        sub = sub_env[sub_env["coeff"] == coeff].copy()
        if sub.empty:
            continue

        # DATA: all seeds as dots
        D = sub["z_dim"].astype(float).to_numpy()
        E = sub["test_Kloss"].astype(float).to_numpy()

        # plot dots WITHOUT legend entry
        # We still want same color for dots + fit. We'll manually grab and reuse it.
        dots = ax.plot(D, E, "o", alpha=0.28, label="_nolegend_")
        color = dots[0].get_color()

        # FIT curve
        popt = fit_scaling_law_log(D, E)
        if popt is not None:
            A_hat, alpha_hat, C_hat = popt

            D_unique = np.unique(D[D > 0])
            D_fit = np.geomspace(D_unique.min(), D_unique.max(), 200)
            E_fit = scaling_model(D_fit, *popt)

            ax.plot(
                D_fit, E_fit, "-", linewidth=2.0, color=color,
                label=f"{coeff_display_name(coeff)}: $\\alpha={alpha_hat:.2f}$"
            )

    ax.set_xlabel("Encode Dimension")
    ax.set_ylabel("Prediction Error (MSE)")
    ax.set_xscale("log", base=2)
    ax.set_yscale("log")
    ax.grid(True, which="both", ls="--", alpha=0.6)
    ax.legend(loc="upper right")

save_path = os.path.join(OUT_DIR, "mlogn_error_vs_encode_dim_fit_by_coeff_clean.pdf")
fig.savefig(save_path, dpi=300)
plt.close(fig)
print(f"🖼️ Saved: {save_path}")

print("\n✅ Done.")



--- Plot: Error vs Encode Dimension (seed dots + fit by coeff) ---
🖼️ Saved: Sep_21/mlogn_error_vs_encode_dim_fit_by_coeff_clean.pdf

✅ Done.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from scipy.optimize import curve_fit

# ============================
# Global Settings (match your style)
# ============================
mpl.rcParams.update({
    "figure.dpi": 150,
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "axes.titlesize": 9,
    "axes.labelsize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
    "lines.linewidth": 1.4,
    "lines.markersize": 5.5,
    "axes.formatter.use_mathtext": True,
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "font.family": "serif",
    "font.serif": ["Times New Roman", "Times", "DejaVu Serif"],
})

# ============================
# Configuration
# ============================
PROJECT_NAME = "Sep_21"
OUT_DIR = PROJECT_NAME
os.makedirs(OUT_DIR, exist_ok=True)

DATA_PATH = f"../log/{PROJECT_NAME}/koopman_correlation_results_log.csv"
assert os.path.exists(DATA_PATH), f"Log CSV not found: {DATA_PATH}"

ENVS  = ["G1", "Go2"]
SEEDS = [17382, 76849, 20965, 84902, 51194]
ENCODE_MULTS = [1, 2, 4, 8, 16]
COEFFS = [1, 5, 10, 20, 40, 100]

# fixed config
LAYER_DEPTHS = [3]
HIDDEN_DIMS  = [256]
USE_RESIDUAL = True
USE_CONTROL_LOSS = True
USE_COV_LOSS = True

# ============================
# Fit helpers
# ============================
def scaling_model(D, A, alpha, C):
    return A * np.power(D, -alpha) + C

def scaling_model_log_fit(D, A, alpha, C):
    pred = scaling_model(D, A, alpha, C)
    pred = np.maximum(pred, 1e-30)  # safety
    return np.log(pred)

def fit_scaling_law_log(D, E):
    """Fit in log-space: minimize || log(E) - log(A D^-alpha + C) ||"""
    D = np.asarray(D, dtype=float)
    E = np.asarray(E, dtype=float)

    mask = (D > 0) & (E > 0)
    D, E = D[mask], E[mask]
    if len(D) < 3:
        return None

    log_D = np.log(D)
    log_E = np.log(E)

    # init: rough slope fit ignoring C
    try:
        slope, intercept = np.polyfit(log_D, log_E, 1)
        alpha_init = -slope
        A_init = np.exp(intercept)
    except Exception:
        alpha_init = 1.0
        A_init = float(np.max(E))

    try:
        popt, _ = curve_fit(
            scaling_model_log_fit,
            D,
            log_E,
            p0=[A_init, alpha_init, 0.0],
            bounds=([0.0, -np.inf, 0.0], [np.inf, np.inf, np.inf]),
            maxfev=20000
        )
        return popt
    except Exception as e:
        print(f"    ⚠️ fit failed: {e}")
        return None

def coeff_display_name(coeff: int) -> str:
    if int(coeff) == 100:
        return "full train sample"
    return f"coeff={int(coeff)}"

# ============================
# Load & Filter
# ============================
log = pd.read_csv(DATA_PATH)

def as_bool(x):
    if isinstance(x, bool):
        return x
    try:
        return bool(int(x))
    except Exception:
        return bool(x)

for c in ["use_residual", "use_control_loss", "use_covariance_loss"]:
    if c in log.columns:
        log[c] = log[c].apply(as_bool)

mask = (
    log["env_name"].isin(ENVS) &
    log["seed"].isin(SEEDS) &
    log["encode_dim_param"].isin(ENCODE_MULTS) &
    log["hidden_layers"].isin(LAYER_DEPTHS) &
    log["hidden_dim"].isin(HIDDEN_DIMS) &
    (log["use_residual"] == USE_RESIDUAL) &
    (log["use_control_loss"] == USE_CONTROL_LOSS) &
    (log["use_covariance_loss"] == USE_COV_LOSS)
)

if "coeff" in log.columns:
    mask &= log["coeff"].isin(COEFFS)

filtered = log[mask].copy()
assert not filtered.empty, "Filtered dataframe is empty — check your mask / columns."

filtered["coeff"] = filtered["coeff"].astype(int)

# We still fit using z_dim, but label x-axis as "Encode Dimension"
filtered["z_dim"] = filtered["state_dim"].astype(float) + filtered["encode_dim"].astype(float)

# ============================
# Color scale (purple -> yellow like your example)
# ============================
coeff_list = sorted(filtered["coeff"].unique())
cmap = plt.cm.viridis  # similar purple->yellow scale
vals = np.linspace(0.05, 0.95, len(coeff_list))
coeff_to_color = {c: cmap(v) for c, v in zip(coeff_list, vals)}

# ============================
# Plot: Error vs Encode Dimension (internally z_dim)
# ============================
print("\n--- Plot: Error vs Encode Dimension (seed dots + dashed fit by coeff) ---")
fig, axes = plt.subplots(1, 2, figsize=(12, 3.8), sharey=False)

for ax, env in zip(axes, ENVS):
    ax.set_title(env)
    sub_env = filtered[filtered["env_name"] == env].copy()

    # build dot-only legend handles (no line in legend)
    legend_handles = []

    for coeff in coeff_list:
        sub = sub_env[sub_env["coeff"] == coeff].copy()
        if sub.empty:
            continue

        color = coeff_to_color[coeff]

        # DATA: all seeds as dots (no legend entry)
        D = sub["z_dim"].astype(float).to_numpy()
        E = sub["test_Kloss"].astype(float).to_numpy()
        ax.plot(
            D, E, "o",
            color=color,
            alpha=0.35,
            label="_nolegend_",
            markeredgewidth=0.0
        )

        # FIT curve (dashed, no legend entry)
        popt = fit_scaling_law_log(D, E)
        if popt is not None:
            A_hat, alpha_hat, C_hat = popt
            D_pos = D[D > 0]
            if D_pos.size > 1:
                D_fit = np.geomspace(D_pos.min(), D_pos.max(), 200)
                E_fit = scaling_model(D_fit, *popt)
                ax.plot(
                    D_fit, E_fit,
                    #linestyle="--",  # dashed like your example
                    linewidth=2.0,
                    color=color,
                    label="_nolegend_"
                )

            # dot-only legend label (optionally includes alpha)
            legend_handles.append(
                Line2D(
                    [0], [0],
                    marker="o",
                    linestyle="None",
                    markersize=6.0,
                    markerfacecolor=color,
                    markeredgecolor=color,
                    label=f"{coeff_display_name(coeff)}: $\\alpha={alpha_hat:.2f}$",
                )
            )
        else:
            # still include in legend even if fit failed (dot only, no alpha)
            legend_handles.append(
                Line2D(
                    [0], [0],
                    marker="o",
                    linestyle="None",
                    markersize=6.0,
                    markerfacecolor=color,
                    markeredgecolor=color,
                    label=f"{coeff_display_name(coeff)}",
                )
            )

    ax.set_xlabel("Encode Dimension")
    ax.set_ylabel("Prediction Error (MSE)")
    ax.set_xscale("log", base=2)
    ax.set_yscale("log")
    ax.grid(True, which="both", ls="--", alpha=0.6)

    if legend_handles:
        ax.legend(handles=legend_handles, loc="upper right", frameon=True)

save_path = os.path.join(OUT_DIR, "mlogn_error_vs_encode_dim_fit_by_coeff_clean.pdf")
fig.savefig(save_path, dpi=300)
plt.close(fig)
print(f"🖼️ Saved: {save_path}")

print("\n✅ Done.")


--- Plot: Error vs Encode Dimension (seed dots + dashed fit by coeff) ---
🖼️ Saved: Sep_21/mlogn_error_vs_encode_dim_fit_by_coeff_clean.pdf

✅ Done.
